In [ ]:
import scipp as sc
import plopp as pp
import scippneutron as scn
import scippnexus as snx
import h5py
from pathlib import Path
import numpy as np

## An simulated sample with incoherent elastic scattering and one phonon mode
The data file specified below contains simulated scattering from a sample with one incoherent elastic scattering mode,
and one phonon mode with instrument-parameter controlled slope, `sound_speed`.

A standard-McStas particle can only scatter from either the elastic mode _or_ phonon,
and this simulation splits the particles equally between the two modes.
A better approach would have been sending significantly more particles to the phonon.

The phonon FCC lattice constant $a=6.56162$ Å was chosen such that its {200} Bragg peaks (not simulated) 
would appear at 90 degree $a_4$ in the $E_f = 3.8$ meV analyzers.

The simulation was conducted as an sample orientation scan with all other parameters fixed.

| Parameter | Value | Notes |
|-----------|-------|-------|
| `sound_speed` / meV Å | 2.5 |  |
| $a_3$ / degree | 0:179 | 1-degree steps, endpoints included |
| $a_4$ / degree | 90. | all simulations have this single detector tank position |
| pulse-shaping chopper opening time  / msec | 0.2 | picked to be a realistic best-case energy-resolution |
| minimum $E_i$ / meV | 2.5 | giving a maximum of ~5.1 meV due to BIFROST's pseudo-white beam |

The simulation was started at approxmately 5:37 UTC on 14. September 2024, 
and required approximately 30 seconds per $a_3$ setting using MPI with 6 nodes.

In [ ]:
datafile = "20240914/BIFROST_20240914T053723.h5"

In [ ]:
from bifrost2409.config import POOCH_DATA_DIR, INTERIM_DATA_DIR
from bifrost2409.dataset import download_datafiles
download_datafiles([datafile])

### Trust the workflow, use the worflow

Now that we trust the process that we performed 'by hand' in the last two notebooks,
we can make use of the same process available through the workflow in abbreviated form
as `bifrost_single`, which _ignores_ any scan information and treats all data as a single setting.

> **Note**: Since the workflow is _intended_ for _real_ data, but our simulations need some manipulations
> you _must_ specify that this datafile `is_simulated`.

In [ ]:
from ess.spectroscopy.indirect import bifrost_single
as_one = bifrost_single(POOCH_DATA_DIR / datafile, is_simulated=True)

We can get an overview of the per-pixel inelastic spectrum (but we plot it here as a function of 10% of a _tube_)

In [ ]:
tube_count = 3 * 9 * 5 * 10
as_one['energy_momentum_events'].bin(energy_transfer=100).hist(
    energy_transfer=sc.linspace(start=-1.7, stop=1.7, num=50, dim='energy_transfer', unit='meV'),
    detector_number=tube_count,
).plot(norm='log')

Converting this data to S(**Q**, E) requires splitting the continuous measurement-time dimension of the data into discrete 
periods with constant settings.
> **Note**: currently, this can be done _exactly_ since the `NXlog` values of simulated parameters are stable and precise.
> The same should be true in most cases for parameter _set points_, so they will be used to segment real data.
>

A different workflow entry-point `bifrost` segments the data (currently limited to constant ($a_3$, $a_4$) pairs),
then calls, effectively, `bifrost_single` per segment before calculating **Q** _in the sample table coordinate system_ 
and combining the data.

_This process is unoptimized and slow -- about 15 minutes on my laptop, so we will store the result to avoid re-creating it unnecessarily_

In [ ]:
from ess.spectroscopy.indirect import bifrost

In [ ]:
targets = ['energy_momentum_events']
target_files = {target: INTERIM_DATA_DIR / f'{Path(datafile).stem}_{target}.h5' for target in targets}
if all(file.exists() for file in target_files.values()):
    from scipp.io import load_hdf5
    objects = {target: load_hdf5(file) for target, file in target_files.items()}
else:
    data = bifrost(POOCH_DATA_DIR / datafile, is_simulated=True)
    objects = {target: data[target] for target in targets}
    for target in targets:
        objects[target].save_hdf5(target_files[target])


In [ ]:
energy_momentum_events = objects['energy_momentum_events']

In [ ]:
def hist_Q_plane(events, energy_transfer_range, q_bins):
    a = events.bin(energy_transfer=energy_transfer_range)
    # Remove coordinates and event coordinates that we're not using:
    for coord in ('a3', 'a4', 'detector_number', 'final_energy'):
        del a.coords[coord]
    for coord in ('event_time_offset', 'event_time_zero', 'frame_time', 'incident_energy', 'lab_momentum_x', 'lab_momentum_z'):
        del a.bins.coords[coord]
    # drop the non-energy_transfer dimensions before binning in Q
    for dim in ('setting', 'event_id'):
        a = a.bins.concat(dim)
    return a.bin(table_momentum_x=q_bins, table_momentum_z=q_bins).hist()['energy_transfer', 0]
    

In [ ]:
hist_Q_plane(energy_momentum_events, sc.array(values=[-0.05, 0.05], dims=['energy_transfer'], unit='meV'), 200).plot(norm='log')

In [ ]:
hist_Q_plane(energy_momentum_events, sc.array(values=[0.3, 0.9], dims=['energy_transfer'], unit='meV'), 200).plot(norm='log')

In [ ]:
hist_Q_plane(energy_momentum_events, sc.array(values=[1., 1.6], dims=['energy_transfer'], unit='meV'), 200).plot(norm='log')

In [ ]:
def hist_E_plane(events, q_x_range, q_bins, e_bins):
    a = events.bin(table_momentum_x=q_x_range)
    # Remove coordinates and event coordinates that we're not using:
    for coord in ('a3', 'a4', 'detector_number', 'final_energy'):
        del a.coords[coord]
    for coord in ('event_time_offset', 'event_time_zero', 'frame_time', 'incident_energy', 'lab_momentum_x', 'lab_momentum_z'):
        del a.bins.coords[coord]
    # drop the non-energy_transfer dimensions before binning in Q
    for dim in ('setting', 'event_id'):
        a = a.bins.concat(dim)
    return a.bin(energy_transfer=e_bins, table_momentum_z=q_bins).hist()['table_momentum_x', 0]
 

In [ ]:
astar = 2 * np.pi / 6.56162

In [ ]:
hist_E_plane(energy_momentum_events, sc.array(values=[2 * astar - 0.2,  2 * astar + 0.2], dims=['table_momentum_x'], unit='1/angstrom'), 200, 50).plot(norm='log')